In [1]:
# Import libraries and setup
from breeze_connect import BreezeConnect
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime, date, timedelta
import functions as f
import ast
from selenium import webdriver
from pyotp import TOTP
import urllib
import json

In [2]:
# Auto-login to ICICI Breeze API
line = ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 8).strip())
date = line['date']

today = datetime.now().date()
today_date = f'{today.day}-{today.month}-{today.year}'

if today_date != date:

    def autologin():

        api_key = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 1).strip())['api_key'])
        api_secret = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 2).strip())['api_secret'])
        password = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 4).strip())['password'])
        userID = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 5).strip())['userID'])
        totp = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 6).strip())['totp'])

        # ----------------- Selenium -----------------

        browser = webdriver.Chrome()
        browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
        browser.implicitly_wait(5)
        breeze = BreezeConnect(api_key=api_key)
        username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
        password_field = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input')

        username.send_keys(userID)
        password_field.send_keys(password)

        # Checkbox
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
        # Click Login Button
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
        time.sleep(2)
        pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')

        totp = TOTP(totp)
        token = totp.now()
        pin.send_keys(token)
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
        time.sleep(1)

        # Check if 'apisession=' exists in the URL
        if 'apisession=' in browser.current_url:
            temp_token = browser.current_url.split('apisession=')[1][:8]
            # Save in Database or text File
            print('temp_token', temp_token)
            f.update_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 7, {'session_key': temp_token})

            breeze.generate_session(api_secret=api_secret, session_token=temp_token)
            print(breeze.get_funds())
        else:
            print("Error: 'apisession=' not found in the URL.")

        browser.quit()

    autologin()

    f.update_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 8, {'date': today_date})

In [3]:
# Initialize Breeze connection with saved credentials
api_key = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 1).strip())['api_key'])
api_secret = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 2).strip())['api_secret'])
session_key = str(ast.literal_eval(f.get_line('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/login_credential.txt', 7).strip())['session_key'])

breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret, session_token=session_key)

print(breeze.get_funds())

{'Success': {'bank_account': '423501000819', 'total_bank_balance': 0.0, 'allocated_equity': 0.0, 'allocated_fno': 0.0, 'allocated_commodity': 0.0, 'allocated_currency': 0.0, 'block_by_trade_equity': 0.0, 'block_by_trade_fno': 0.0, 'block_by_trade_commodity': 0.0, 'block_by_trade_currency': 0.0, 'block_by_trade_balance': 0.0, 'unallocated_balance': '225901.1'}, 'Status': 200, 'Error': None}


________________________ Data operations ________________________

In [4]:
# Set pandas display options
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
# nifty_50_componants_data_from_tradingview.csv
nifty_comprehensive_df = pd.read_csv("D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv")


In [6]:
# Read Master_list file of icici
df = pd.read_csv(r"D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_ICICI_via_API\master_list_files\NSEScripMaster.csv")
df = df[df['Series'] == 'EQ']
df

,Token,ShortName,Series,CompanyName,ticksize,Lotsize,DateOfListing,DateOfDeListing,IssuePrice,FaceValue,ISINCode,52WeeksHigh,52WeeksLow,LifeTimeHigh,LifeTimeLow,HighDate,LowDate,Symbol,InstrumentType,PermittedToTrade,IssueCapital,WarningPercent,FreezePercent,CreditRating,IssueRate,IssueStartDate,InterestPaymentDate,IssueMaturityDate,BoardLotQty,Name,ListingDate,ExpulsionDate,ReAdmissionDate,RecordDate,ExpiryDate,NoDeliveryStartDate,NoDeliveryEndDate,MFill,AON,ParticipantInMarketIndex,BookClsStartDate,BookClsEndDate,EGM,AGM,Interest,Bonus,Rights,Dividends,LocalUpdateDateTime,DeleteFlag,Remarks,NormalMarketStatus,OddLotMarketStatus,SpotMarketStatus,AuctionMarketStatus,NormalMarketEligibility,OddLotlMarketEligibility,SpotMarketEligibility,AuctionlMarketEligibility,MarginPercentage,ExchangeCode
1,0,ALFLAV,EQ,ALFA LAVAL (INDIA) LIMITED,0.01,1,NaN,NaN,0,10.0,INE427A01017,0.00,0.00,3949.95,73.00,NaN,NaN,NaN,0,0,0,4736.30,3157.60,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,ALFALAVAL
3,0,ANDBAN,EQ,ANDHRA BANK,0.01,1,NaN,NaN,0,10.0,INE434A01013,33.65,7.65,189.90,7.30,NaN,NaN,NaN,0,0,0,10.90,7.30,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,ANDHRABANK
4,0,ARITHR,EQ,ARIHANT THREADS LTD,0.01,1,NaN,NaN,0,10.0,INE747A01018,0.00,0.00,9.00,0.30,NaN,NaN,NaN,0,0,0,2.55,1.75,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,ARIHANTRED
5,0,BANRAJ,EQ,THE BANK OF RAJASTHAN LTD,0.01,1,NaN,NaN,0,10.0,INE320A01014,0.00,0.00,213.95,5.33,NaN,NaN,NaN,0,0,0,233.90,191.35,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,BANKRAJAS
6,0,BELCER,EQ,BELL CERAMICS LIMITED,0.01,1,NaN,NaN,0,10.0,INE403A01026,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0,0,0,10.40,6.95,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,BELCERAMIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5060,NTPOFS,NTPOFS,EQ,NTPC LTD OFS,0.01,1,NaN,NaN,0,10.0,INE000000014,0.00,0.00,290.75,70.00,NaN,NaN,NaN,0,0,0,0.00,0.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NTPOFS
5062,POWOFS,POWOFS,EQ,POWER FINANCE CORPO LTD OFS,0.01,1,NaN,NaN,0,10.0,INE000000097,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0,0,0,0.00,0.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,POWOFS
5063,RUROFS,RUROFS,EQ,RURAL ELECTRIFICATION CORP OFS,0.01,1,NaN,NaN,0,10.0,INE000000093,0.00,0.00,413.80,53.00,NaN,NaN,NaN,0,0,0,0.00,0.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,RUROFS
5068,X25903X,DALREF,EQ,DALMIA REFRACTORIES,0.01,1,NaN,NaN,0,10.0,INE200F01017,170.00,49.80,170.00,49.80,NaN,NaN,NaN,0,0,0,135.00,90.00,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,DALMIAREF


In [7]:
# # ----------------------------------- find stock in masterlist file -----------------------------------#

n = 0

# Nifty 50 symbols to find
nifty_50 = nifty_comprehensive_df['Symbol'].values.tolist()
nifty_50 = [symbol.replace("_", "-") for symbol in nifty_50]

x = 0
# Fix: Iterate over the actual values in the list, not indices
for stock in nifty_50:
    if stock in df["ExchangeCode"].values:
        x += 1
        n += 1
        # a = print(x, stock, "\tFound") if len(stock) > 4 else print(x, stock, "\t\tFound")
    else:
        x += 1
        print(x, stock, "\tNot found")

print(f"Total {n} Nifty 50 stocks found in the DataFrame.")

Total 50 Nifty 50 stocks found in the DataFrame.


In [8]:
filtered_masterlist = df[df['ExchangeCode'].isin(nifty_50)].reset_index(drop=True)
nifty50_stockcode = filtered_masterlist['ShortName'].to_list()
print(nifty50_stockcode)

['BHAAIR', 'MARUTI', 'LARTOU', 'ULTCEM', 'TCS', 'NTPC', 'JSWSTE', 'GRASIM', 'HDFBAN', 'HERHON', 'TECMAH', 'HINDAL', 'HINLEV', 'POWGRI', 'ADAPOR', 'APOHOS', 'INFTEC', 'ITC', 'BAAUTO', 'BAFINS', 'NESIND', 'JIOFIN', 'KOTMAH', 'TRENT', 'MAHMAH', 'COALIN', 'SBILIF', 'ASIPAI', 'ONGC', 'ADAENT', 'RELIND', 'STABAN', 'BAJFI', 'SUNPHA', 'TATGLO', 'TATMOT', 'TATSTE', 'TITIND', 'WIPRO', 'BHAELE', 'SHRTRA', 'HDFSTA', 'ICIBAN', 'ZOMLIM', 'INDBA', 'AXIBAN', 'CIPLA', 'HCLTEC', 'DRREDD', 'EICMOT']


In [9]:
# Download Nifty 50 components data and save to folder (Same as Fyers API structure)
import os
import time

# Create directory for saving data
save_dir = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty50_daily_data"
os.makedirs(save_dir, exist_ok=True)

# Date range for download
date_range = {
    "exchange_code": "NSE",
    "interval": "1day",
    "from_date": "2025-08-10T09:15:00.000Z", 
    "to_date": "2025-09-15T15:30:00.000Z"
}

print(f"Starting download of {len(nifty50_stockcode)} Nifty 50 stocks...")
print(f"Date range: {date_range['from_date']} to {date_range['to_date']}")
print(f"Saving to: {save_dir}")

successful_downloads = 0
failed_downloads = []

for i, symbol in enumerate(nifty50_stockcode, 1):
    try:
        # Prepare data for API call
        data = {
            "stock_code": symbol,
            **date_range
        }
        
        # Make API call
        response = breeze.get_historical_data_v2(**data)
        
        if response and response.get('Success'):
            # Create DataFrame
            df = pd.DataFrame(response['Success'])
            df['datetime'] = pd.to_datetime(df['datetime'])
            df['symbol'] = symbol
            df = df[['symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume']]
            
            # Save to CSV
            filename = f"{symbol}_daily_data.csv"
            filepath = os.path.join(save_dir, filename)
            df.to_csv(filepath, index=False)
            
            successful_downloads += 1
            print(f"{i:2d}. {symbol:20s} - Downloaded {len(df)} records -> {filename}")
            
        else:
            failed_downloads.append(symbol)
            print(f"{i:2d}. {symbol:20s} - Failed: {response.get('Error', 'Unknown error')}")
        
        # Add small delay to avoid rate limiting
        time.sleep(0.1)
        
    except Exception as e:
        failed_downloads.append(symbol)
        print(f"{i:2d}. {symbol:20s} - Error: {str(e)}")

print(f"\nDownload Summary:")
print(f"Successful: {successful_downloads}")
print(f"Failed: {len(failed_downloads)}")
if failed_downloads:
    print(f"Failed symbols: {failed_downloads}")
print(f"Data saved in: {save_dir}")

Starting download of 50 Nifty 50 stocks...
Date range: 2025-08-10T09:15:00.000Z to 2025-09-15T15:30:00.000Z
Saving to: D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty50_daily_data
 1. BHAAIR               - Downloaded 24 records -> BHAAIR_daily_data.csv
 2. MARUTI               - Downloaded 24 records -> MARUTI_daily_data.csv
 3. LARTOU               - Downloaded 24 records -> LARTOU_daily_data.csv
 4. ULTCEM               - Downloaded 24 records -> ULTCEM_daily_data.csv
 5. TCS                  - Downloaded 24 records -> TCS_daily_data.csv
 6. NTPC                 - Downloaded 24 records -> NTPC_daily_data.csv
 7. JSWSTE               - Downloaded 24 records -> JSWSTE_daily_data.csv
 8. GRASIM               - Downloaded 24 records -> GRASIM_daily_data.csv
 9. HDFBAN               - Downloaded 24 records -> HDFBAN_daily_data.csv
10. HERHON               - Downloaded 24 records -> HERHON_daily_data.csv
11. TECMAH               - Downloaded 2

In [10]:
# Sum all stocks volume by datetime
import glob

# Path to the folder containing individual stock CSV files
data_folder = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty50_daily_data"

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, "*_daily_data.csv"))

print(f"Found {len(csv_files)} CSV files to process...")

# Initialize list to store all dataframes
all_dataframes = []

# Read each CSV file and collect dataframes
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['datetime'])
        # Keep only datetime and volume columns with symbol info
        df_volume = df[['datetime', 'volume', 'symbol']].copy()
        all_dataframes.append(df_volume)
        # print(f"Loaded: {os.path.basename(file_path)} - {len(df)} records")
    except Exception as e:
        print(f"Error loading {os.path.basename(file_path)}: {str(e)}")

if all_dataframes:
    # Combine all dataframes
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Group by datetime and sum volumes
    volume_summary = combined_df.groupby('datetime').agg({
        'volume': 'sum',
        'symbol': 'count'  # Count how many stocks contributed to each date
    }).reset_index()
    
    # Rename columns for clarity
    volume_summary.rename(columns={'symbol': 'stock_count'}, inplace=True)
    
    # Sort by datetime
    volume_summary = volume_summary.sort_values('datetime').reset_index(drop=True)
    
    print(f"\nVolume Summary (Total across all Nifty 50 stocks):")
    print("=" * 60)
    print(volume_summary.to_string(index=False))
    
else:
    print("No data files found to process!")

Found 50 CSV files to process...

Volume Summary (Total across all Nifty 50 stocks):
  datetime    volume  stock_count
2025-08-11 240227122           50
2025-08-12 236273615           50
2025-08-13 234556839           50
2025-08-14 268889089           50
2025-08-18 360161897           50
2025-08-19 251248126           50
2025-08-20 285749934           50
2025-08-21 224840261           50
2025-08-22 209065830           50
2025-08-25 211762988           50
2025-08-26 749829875           50
2025-08-28 320809024           50
2025-08-29 324025257           50
2025-09-01 227573535           50
2025-09-02 270895089           50
2025-09-03 338074126           50
2025-09-04 309900434           50
2025-09-05 208975977           50
2025-09-08 211606036           50
2025-09-09 225596970           50
2025-09-10 241871866           50
2025-09-11 222958571           50
2025-09-12 224198179           50
2025-09-15 184007609           50


In [11]:
# Convert volume to millions for better readability (Same as Fyers API)
volume_summary['volume'] = round((volume_summary['volume'] / 1000000), 2)
volume_summary

,datetime,volume,stock_count
0,2025-08-11,240.23,50
1,2025-08-12,236.27,50
2,2025-08-13,234.56,50
3,2025-08-14,268.89,50
4,2025-08-18,360.16,50
5,2025-08-19,251.25,50
6,2025-08-20,285.75,50
7,2025-08-21,224.84,50
8,2025-08-22,209.07,50
9,2025-08-25,211.76,50


----------------------- Testing cell ------------------------

In [16]:
# Create tvml_df with both TradingView and ICICI symbol formats
# Read TradingView Nifty 50 components file
tv_df = pd.read_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv')

# Extract symbols and company names
symbols = tv_df['Symbol'].tolist()
company_names = tv_df['Company_Name'].tolist()

# Handle special case for BAJAJ_AUTO
symbols = [symbol.replace("_", "-") for symbol in symbols]

# Get ExchangeCode from filtered_masterlist for each symbol
exchange_codes = []
for symbol in symbols:
    matching_row = filtered_masterlist[filtered_masterlist['ExchangeCode'] == symbol]
    if not matching_row.empty:
        exchange_codes.append(matching_row['ShortName'].iloc[0])
    else:
        exchange_codes.append(None)

# Create tvml_df with all three columns
tvml_df = pd.DataFrame({
    'tv_symbol': symbols,
    'company_names': company_names,
    'exchange_code': exchange_codes
})

tvml_df.head()

,tv_symbol,company_names,exchange_code
0,RELIANCE,Reliance Industries Limited,RELIND
1,HDFCBANK,HDFC Bank Limited,HDFBAN
2,TCS,Tata Consultancy Services Limited,TCS
3,BHARTIARTL,Bharti Airtel Limited,BHAAIR
4,ICICIBANK,ICICI Bank Limited,ICIBAN


In [13]:
# Create n5v_df with symbol and volume for date 2025-08-22 (Same as Fyers cell-14)
import pandas as pd
import glob
import os

# Path to the folder containing individual stock CSV files
data_folder = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty50_daily_data"

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, "*_daily_data.csv"))

# Target date
target_date = '2025-08-22'

# Initialize list to store data for target date
data_for_date = []

# Read each CSV file and extract data for target date
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        # Filter for target date
        target_data = df[df['datetime'].dt.date == pd.to_datetime(target_date).date()]
        
        if not target_data.empty:
            symbol = target_data['symbol'].iloc[0]
            volume = target_data['volume'].iloc[0]
            data_for_date.append({'symbol': symbol, 'volume': volume})
            
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {str(e)}")

# Create n5v_df
n5v_df = pd.DataFrame(data_for_date)

# Sort by symbol for better readability
n5v_df = n5v_df.sort_values('symbol').reset_index(drop=True)

print(f"n5v_df - Nifty 50 Volume Data for {target_date}")
print("=" * 50)
print(n5v_df.to_string(index=False))
print(f"\nTotal stocks: {len(n5v_df)}")
print(f"Total volume: {n5v_df['volume'].sum()/1000000:,}")

ea = n5v_df

n5v_df - Nifty 50 Volume Data for 2025-08-22
symbol   volume
ADAENT   580396
ADAPOR  1207068
APOHOS   835349
ASIPAI   522264
AXIBAN  3839423
BAAUTO   368799
BAFINS   707392
 BAJFI  6412686
BHAAIR  3239536
BHAELE 15162276
 CIPLA  1563833
COALIN  4679162
DRREDD  1224169
EICMOT   272835
GRASIM   670948
HCLTEC  2796281
HDFBAN  9776050
HDFSTA  1329384
HERHON  1355565
HINDAL  5611112
HINLEV  1219535
ICIBAN  6543030
 INDBA  2478766
INFTEC  5512716
   ITC 13040795
JIOFIN  8736355
JSWSTE  1948451
KOTMAH  1785364
LARTOU  1058415
MAHMAH  2505056
MARUTI   455792
NESIND   920097
  NTPC  8523112
  ONGC  5412275
POWGRI  6794623
RELIND  5771445
SBILIF   604995
SHRTRA  3491489
STABAN  5079645
SUNPHA  2094453
TATGLO   392774
TATMOT  6403248
TATSTE 16022867
   TCS  1579669
TECMAH   966526
TITIND   690298
 TRENT   443009
ULTCEM   226657
 WIPRO 13460588
ZOMLIM 22749257

Total stocks: 50
Total volume: 209.06583


In [19]:
# First verify symbol matching between ea and tvml_df
print("Symbol Matching Verification:")
print("=" * 40)

# Get unique symbols from both DataFrames
ea_symbols = set(ea['symbol'].tolist())
tvml_symbols = set(tvml_df['exchange_code'].tolist())

# Find matches and mismatches
matched_symbols = ea_symbols.intersection(tvml_symbols)
ea_only = ea_symbols - tvml_symbols
tvml_only = tvml_symbols - ea_symbols

print(f"EA symbols: {len(ea_symbols)}")
print(f"TVML symbols: {len(tvml_symbols)}")
print(f"Matched symbols: {len(matched_symbols)}")
print(f"EA only symbols: {len(ea_only)}")
print(f"TVML only symbols: {len(tvml_only)}")

if ea_only:
    print(f"\nSymbols in EA but not in TVML: {sorted(ea_only)}")
if tvml_only:
    print(f"\nSymbols in TVML but not in EA: {sorted(tvml_only)}")

print(f"\nMatched symbols: {sorted(matched_symbols)}")

# After verification, add company names to ea DataFrame
# Note: Using 'company_names' (plural) as per tvml_df column name
ea_with_names = ea.merge(tvml_df[['exchange_code', 'company_names']], 
                        left_on='symbol', 
                        right_on='exchange_code', 
                        how='left')

# Drop the duplicate breeze_symbol column and reorder columns
ea_with_names = ea_with_names[['symbol', 'company_names', 'volume']].copy()

# Rename column to singular for consistency
ea_with_names.rename(columns={'company_names': 'company_name'}, inplace=True)

print(f"\nEA DataFrame with Company Names:")
print("=" * 50)
ea_with_names.head()

Symbol Matching Verification:
EA symbols: 50
TVML symbols: 50
Matched symbols: 50
EA only symbols: 0
TVML only symbols: 0

Matched symbols: ['ADAENT', 'ADAPOR', 'APOHOS', 'ASIPAI', 'AXIBAN', 'BAAUTO', 'BAFINS', 'BAJFI', 'BHAAIR', 'BHAELE', 'CIPLA', 'COALIN', 'DRREDD', 'EICMOT', 'GRASIM', 'HCLTEC', 'HDFBAN', 'HDFSTA', 'HERHON', 'HINDAL', 'HINLEV', 'ICIBAN', 'INDBA', 'INFTEC', 'ITC', 'JIOFIN', 'JSWSTE', 'KOTMAH', 'LARTOU', 'MAHMAH', 'MARUTI', 'NESIND', 'NTPC', 'ONGC', 'POWGRI', 'RELIND', 'SBILIF', 'SHRTRA', 'STABAN', 'SUNPHA', 'TATGLO', 'TATMOT', 'TATSTE', 'TCS', 'TECMAH', 'TITIND', 'TRENT', 'ULTCEM', 'WIPRO', 'ZOMLIM']

EA DataFrame with Company Names:


,symbol,company_name,volume
0,ADAENT,Adani Enterprises Limited,580396
1,ADAPOR,Adani Ports & Special Economic Zone Ltd,1207068
2,APOHOS,Apollo Hospitals Enterprise Limited,835349
3,ASIPAI,Asian Paints Ltd.,522264
4,AXIBAN,Axis Bank Limited,3839423


In [18]:
# save ea_with_names in D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_ICICI_via_API
ea_with_names.to_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/ea_with_names.csv', index=False)

___________________ TradingView Data extracting ___________________

In [ ]:
# Diagnostic and error handling approach
import sys
import subprocess

print("Python version:", sys.version)
print("Python executable:", sys.executable)

# Check if we can install packages
try:
    result = subprocess.run([sys.executable, "-m", "pip", "--version"], 
                          capture_output=True, text=True)
    print("Pip version:", result.stdout.strip())
except Exception as e:
    print("Pip error:", e)

# Try multiple installation methods
methods = [
    f"{sys.executable} -m pip install yfinance",
    "pip install yfinance", 
    f"{sys.executable} -m pip install yfinance --user",
    "conda install -c conda-forge yfinance"
]

for i, method in enumerate(methods, 1):
    print(f"\nMethod {i}: {method}")
    try:
        result = subprocess.run(method.split(), capture_output=True, text=True, timeout=60)
        if result.returncode == 0:
            print("✓ Installation successful!")
            break
        else:
            print("✗ Installation failed:", result.stderr[:200])
    except subprocess.TimeoutExpired:
        print("✗ Installation timed out")
    except Exception as e:
        print("✗ Error:", str(e))

# Test import
print("\nTesting import...")
try:
    import yfinance as yf
    print("✓ yfinance imported successfully!")
    
    # Test basic functionality
    print("Testing basic download...")
    ticker = yf.Ticker("AAPL")  # Use a simple US stock first
    info = ticker.info
    print("✓ Basic functionality works!")
    
except ImportError as e:
    print("✗ Import failed:", e)
    print("Possible solutions:")
    print("1. Restart your Jupyter kernel")
    print("2. Try running: !pip install yfinance")
    print("3. Check if you're in a virtual environment")
    
except Exception as e:
    print("✗ Function test failed:", e)

In [ ]:
# Alternative: Manual data collection method (if yfinance fails)
print("If yfinance doesn't work, here are alternative approaches:")

print("\n1. MANUAL APPROACH:")
print("   - Go to finance.yahoo.com")
print("   - Search for stocks with .NS suffix (e.g., RELIANCE.NS)")
print("   - Click 'Historical Data' tab")
print("   - Select date range and download CSV")

print("\n2. REQUESTS-BASED APPROACH:")
try:
    import requests
    import json
    
    print("✓ Using requests library for direct API calls")
    
    # Example Yahoo Finance API call (this is what yfinance uses internally)
    symbol = "RELIANCE.NS"
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"✓ Successfully accessed Yahoo Finance API for {symbol}")
        data = response.json()
        print("✓ Data received - can parse manually if needed")
    else:
        print(f"✗ API call failed: {response.status_code}")
        
except Exception as e:
    print(f"✗ Requests approach failed: {e}")

print("\n3. EXISTING ICICI DATA:")
print("   - You already have ICICI Breeze data working")
print("   - This might be sufficient for your trading needs")
print("   - ICICI data is from actual broker, often more reliable")

print("\n4. CSV UPLOAD METHOD:")
print("   - Download data manually from TradingView/Yahoo")
print("   - Save as CSV files")
print("   - Load using pandas.read_csv()")

print("\nPlease run the diagnostic cell first and share the output.")
print("This will help identify the exact issue preventing data download.")

In [ ]:
# Option 2: Download all Nifty 50 stocks from TradingView using existing symbol list
import os

# Create directory for TradingView data
tv_save_dir = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/tradingview_data"
os.makedirs(tv_save_dir, exist_ok=True)

# Use TradingView symbols from your existing CSV
tv_symbols = tvml_df['tv_symbol'].tolist()  # Using your existing symbol mapping
print(f"Downloading TradingView data for {len(tv_symbols)} Nifty 50 stocks...")

tv_successful = 0
tv_failed = []

for i, symbol in enumerate(tv_symbols, 1):
    try:
        # Download data from TradingView
        data = tv.get_hist(symbol=symbol, exchange='NSE', interval=Interval.in_daily, n_bars=500)
        
        if data is not None and not data.empty:
            # Add symbol column for consistency
            data['symbol'] = symbol
            
            # Save to CSV
            filename = f"{symbol}_tv_data.csv"
            filepath = os.path.join(tv_save_dir, filename)
            data.to_csv(filepath)
            
            tv_successful += 1
            print(f"{i:2d}. {symbol:15s} - Downloaded {len(data)} records -> {filename}")
        else:
            tv_failed.append(symbol)
            print(f"{i:2d}. {symbol:15s} - No data received")
            
        # Small delay to avoid rate limiting
        time.sleep(0.5)
        
    except Exception as e:
        tv_failed.append(symbol)
        print(f"{i:2d}. {symbol:15s} - Error: {str(e)}")

print(f"\nTradingView Download Summary:")
print(f"Successful: {tv_successful}")
print(f"Failed: {len(tv_failed)}")
if tv_failed:
    print(f"Failed symbols: {tv_failed}")
print(f"Data saved in: {tv_save_dir}")